### Для входа нужны
- users.csv
- data_mos.xlsx

### Выход:
- _users_features.csv

In [1]:
import numpy as np
import pandas as pd

In [2]:
df_users = pd.read_csv('users.csv')

In [3]:
df_users=df_users.rename(columns={'уникальный номер':'user_id','пол':'sex','дата рождения':'birthday','адрес проживания':'address'})


### Добавляем возраст

In [4]:
df_users["age"] = (pd.to_datetime("today") - pd.to_datetime(df_users["birthday"]))
df_users['age'] = (df_users['age'].dt.days / 365).astype(int)

### Добавляем пол

In [5]:
df_users['sex'].replace(('Женщина', 'Мужчина'), ('Female', 'Male'), inplace=True)

In [6]:
df_users.drop(columns=['дата создание личного дела','birthday'], inplace=True)

In [7]:
display(df_users)

,user_id,sex,address,age
0,101391104,Female,"город москва, константинова, дом 30",63
1,101437383,Female,"г. москва, большая серпуховская улица, дом 40,...",64
2,101351338,Female,"город москва, летчика грицевца, дом 10",66
3,101354578,Female,"город москва, лобачевского, дом 76",84
4,101411997,Female,"г. москва, улица вертолётчиков, дом 13",72
...,...,...,...,...
52329,101389206,Female,"город москва, шоссе энтузиастов, дом 48/1",65
52330,101355680,Female,"город москва, аргуновская, дом 16, корпус 1",77
52331,101414122,Male,"город москва, улица генерала белова, дом 41",72
52332,101428464,Female,"г. москва, город зеленоград, корпус 828б",76


### Добавляем улицу проживания

In [8]:
def short_address(address):
    try:
        start = address.find('москва')
        address = address[start+7:]
        address = address[:address.find(',')]
    except:
        address = 'москва'
    address = address.replace('улица','').strip().lower()
    address = address.replace('ул.','').strip().lower()
    return address


df_users['user_address'] = df_users['address'].apply(short_address)

In [9]:
display(df_users)

,user_id,sex,address,age,user_address
0,101391104,Female,"город москва, константинова, дом 30",63,константинова
1,101437383,Female,"г. москва, большая серпуховская улица, дом 40,...",64,большая серпуховская
2,101351338,Female,"город москва, летчика грицевца, дом 10",66,летчика грицевца
3,101354578,Female,"город москва, лобачевского, дом 76",84,лобачевского
4,101411997,Female,"г. москва, улица вертолётчиков, дом 13",72,вертолётчиков
...,...,...,...,...,...
52329,101389206,Female,"город москва, шоссе энтузиастов, дом 48/1",65,шоссе энтузиастов
52330,101355680,Female,"город москва, аргуновская, дом 16, корпус 1",77,аргуновская
52331,101414122,Male,"город москва, улица генерала белова, дом 41",72,генерала белова
52332,101428464,Female,"г. москва, город зеленоград, корпус 828б",76,город зеленоград


### Добавляем координаты

In [10]:
mid = '(55.7498, 37.5371)' # Башня Федерация

# Получаем координаты
df_uniq_streets_coords = pd.read_csv('uniq_streets_coords.csv')
df_uniq_streets_coords = df_uniq_streets_coords[['для координат','Coords']]
df_uniq_streets_coords['для координат']= df_uniq_streets_coords['для координат'].str.lower()
df_uniq_streets_coords['Coords'].fillna(mid, inplace=True)

streets_dict = dict(zip(df_uniq_streets_coords['для координат'], df_uniq_streets_coords['Coords']))

In [ ]:
def match_street_coords(street):  
    res = []
    for i in streets_dict.keys():
        if(i.find(street) != -1):
            res.append(streets_dict[i])    
    if len(res) > 0:
        return res[0]
    else:
        return '(55.7498, 37.5371)'
    
    
split_data = df_users['user_address'].apply(match_street_coords).str.strip(')').str.strip('(').str.split(',')
df_users['user_lat'] = split_data.apply(lambda x: x[0]).astype(float)
df_users['user_long'] = split_data.apply(lambda x: x[1]).astype(float)

df_users['user_lat'].fillna(55.7498, inplace=True)
df_users['user_long'].fillna(37.5371, inplace=True)

display(df_users)

### Сохраняем результат

In [ ]:
df_users.drop(columns=['address']).to_csv('_users_features.csv')